In [1]:
# Dependencies
import pandas as pd
import hvplot.pandas
import requests
import matplotlib.pyplot as plt
import numpy as np
import json
from census import Census

In [16]:
# The correct encoding must be used to read the CSV in pandas
kyoto_df = pd.read_csv("Resources/Kyoto_Restaurant_Info.csv")

# Display sample data
kyoto_df


,Unnamed: 0,Name,JapaneseName,Station,FirstCategory,SecondCategory,DinnerPrice,LunchPrice,TotalRating,DinnerRating,LunchRating,ReviewNum,Lat,Long
0,1,Orudeidainingurajou,オールデイダイニング ラジョウ,Kyoto,Buffet style,Cafe,￥4000～￥4999,￥2000～￥2999,3.39,3.20,3.38,56,34.986175,135.761314
1,2,Steak Frites Gaspard zinzin,ステックフリット ガスパール ザンザン,Karasuma,Bistro,Steak,￥3000～￥3999,￥1000～￥1999,3.18,3.06,3.33,70,35.005826,135.759857
2,3,KAZUMA,和馬,Sanjo,Izakaya (Tavern),Japanese food (other),￥3000～￥3999,NaN,3.28,3.28,NaN,7,35.006583,135.770247
3,4,okonomiyakiteppanyakimiki,お好み焼き 鉄板焼き 三喜,Tambaguchi,Okonomiyaki,Izakaya (Tavern),￥3000～￥3999,NaN,3.14,3.14,NaN,16,34.995924,135.748267
4,5,Shaofeiyan,小肥羊 京都河原町店,Kyoto Shiyakusho Mae,Chinese hot pot / fire pot,Chinese,￥4000～￥4999,￥1000～￥1999,3.16,3.16,3.05,23,35.007901,135.768202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,891,Waraku,和楽,Fukuchiyama,Izakaya (Tavern),Kaiseki (Traditional Japanese),￥2000～￥2999,～￥999,3.00,3.06,3.00,12,35.298917,135.118341
891,892,YEBISU BAR,エビスバー 京都ヨドバシ店,Kyoto,Beer bar,Bar,￥3000～￥3999,～￥999,3.08,3.07,3.08,70,34.988132,135.759215
892,893,Ibushiginkazuya,燻吟 かず家,Kawaramachi,Bar,Nihonshu (Japanese sake),￥3000～￥3999,NaN,3.36,3.36,3.05,11,35.006449,135.769653
893,894,Tenichi,天一 京都伊勢丹店,Kyoto,Tempura,Ten-don (Tempura Bowl),￥6000～￥7999,￥2000～￥2999,3.51,3.32,3.53,52,34.985887,135.757672


In [17]:
# Delete extraneous column
kyoto_df = kyoto_df.drop(columns = ["JapaneseName", "DinnerRating", "LunchRating"])
kyoto_df.head()

,Unnamed: 0,Name,Station,FirstCategory,SecondCategory,DinnerPrice,LunchPrice,TotalRating,ReviewNum,Lat,Long
0,1,Orudeidainingurajou,Kyoto,Buffet style,Cafe,￥4000～￥4999,￥2000～￥2999,3.39,56,34.986175,135.761314
1,2,Steak Frites Gaspard zinzin,Karasuma,Bistro,Steak,￥3000～￥3999,￥1000～￥1999,3.18,70,35.005826,135.759857
2,3,KAZUMA,Sanjo,Izakaya (Tavern),Japanese food (other),￥3000～￥3999,NaN,3.28,7,35.006583,135.770247
3,4,okonomiyakiteppanyakimiki,Tambaguchi,Okonomiyaki,Izakaya (Tavern),￥3000～￥3999,NaN,3.14,16,34.995924,135.748267
4,5,Shaofeiyan,Kyoto Shiyakusho Mae,Chinese hot pot / fire pot,Chinese,￥4000～￥4999,￥1000～￥1999,3.16,23,35.007901,135.768202


In [23]:
kyoto_df['Category'] = kyoto_df[['FirstCategory', 'SecondCategory']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# Drop the original columns
kyoto_df = kyoto_df.drop(['FirstCategory', 'SecondCategory'], axis=1)

kyoto_df

,Unnamed: 0,Name,Station,DinnerPrice,LunchPrice,TotalRating,ReviewNum,Lat,Long,Category
0,1,Orudeidainingurajou,Kyoto,￥4000～￥4999,￥2000～￥2999,3.39,56,34.986175,135.761314,Buffet style Cafe
1,2,Steak Frites Gaspard zinzin,Karasuma,￥3000～￥3999,￥1000～￥1999,3.18,70,35.005826,135.759857,Bistro Steak
2,3,KAZUMA,Sanjo,￥3000～￥3999,NaN,3.28,7,35.006583,135.770247,Izakaya (Tavern) Japanese food (other)
3,4,okonomiyakiteppanyakimiki,Tambaguchi,￥3000～￥3999,NaN,3.14,16,34.995924,135.748267,Okonomiyaki Izakaya (Tavern)
4,5,Shaofeiyan,Kyoto Shiyakusho Mae,￥4000～￥4999,￥1000～￥1999,3.16,23,35.007901,135.768202,Chinese hot pot / fire pot Chinese
...,...,...,...,...,...,...,...,...,...,...
890,891,Waraku,Fukuchiyama,￥2000～￥2999,～￥999,3.00,12,35.298917,135.118341,Izakaya (Tavern) Kaiseki (Traditional Japanese)
891,892,YEBISU BAR,Kyoto,￥3000～￥3999,～￥999,3.08,70,34.988132,135.759215,Beer bar Bar
892,893,Ibushiginkazuya,Kawaramachi,￥3000～￥3999,NaN,3.36,11,35.006449,135.769653,Bar Nihonshu (Japanese sake)
893,894,Tenichi,Kyoto,￥6000～￥7999,￥2000～￥2999,3.51,52,34.985887,135.757672,Tempura Ten-don (Tempura Bowl)


In [18]:

kyoto_df.count()

Unnamed: 0        895
Name              895
Station           895
FirstCategory     895
SecondCategory    867
DinnerPrice       895
LunchPrice        484
TotalRating       895
ReviewNum         895
Lat               895
Long              895
dtype: int64

In [24]:

kyoto_df.describe()

,Unnamed: 0,TotalRating,ReviewNum,Lat,Long
count,895.000000,895.000000,895.000000,895.000000,895.000000
mean,448.000000,3.212480,27.080447,35.005287,135.755774
std,258.508543,0.215588,58.064708,0.062367,0.056629
min,1.000000,3.000000,1.000000,34.715818,135.094284
25%,224.500000,3.050000,7.000000,34.995639,135.757501
50%,448.000000,3.090000,15.000000,35.004454,135.762578
75%,671.500000,3.420000,30.000000,35.008488,135.770655
max,895.000000,4.190000,1319.000000,35.735303,135.831994
